In [1]:
from stable_diffusion.constants import IODirectoryTree

In [2]:
path_tree = IODirectoryTree(base_directory='./')
path_tree

ModelsPathTree(base_directory=./)
                - latent_diffusion: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/latent_diffusion/latent_diffusion.ckpt
                
                -- autoencoder: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/autoencoder/autoencoder.ckpt
                --- encoder: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/autoencoder/encoder.ckpt
                --- decoder: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/autoencoder/decoder.ckpt
                
                -- embedder: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/clip_text_embedder/clip_embedder.ckpt
                --- tokenizer: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/clip_text_embedder/clip_tokenizer.ckpt
                --- transformer: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/clip_text_embedder/clip_transformer.ckpt
                
                -- unet: /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/unet/unet.ckpt

                - image_encoder: /h

In [3]:
from stable_diffusion.constants import UNET_PATH
from stable_diffusion.utils.utils import get_device
from stable_diffusion.utils.utils import SectionManager as section
from stable_diffusion.utils.model import initialize_unet
from safetensors.torch import load_file, load_model
# from labml.monit import section
from time import time
import torch

In [4]:
from stable_diffusion.model.unet import UNetModel

def initialize_unet(device = None, 
                    in_channels=4,
                    out_channels=4,
                    channels=320,
                    attention_levels=[0, 1, 2],
                    n_res_blocks=2,
                    channel_multipliers=[1, 2, 4, 4],
                    n_heads=8,
                    tf_layers=1,
                    d_cond=768) -> UNetModel:

    # Initialize the U-Net
    device = get_device(device)
    with section('U-Net initialization') as sp:
        t0 = time()
        unet_model = UNetModel(in_channels=in_channels,
                                out_channels=out_channels,
                                channels=channels,
                                attention_levels=attention_levels,
                                n_res_blocks=n_res_blocks,
                                channel_multipliers=channel_multipliers,
                                n_heads=n_heads,
                                tf_layers=tf_layers,
                                d_cond=d_cond).to(device)
            # unet_model.save()
            # torch.save(unet_model, UNET_PATH)
        tf = time() - t0
        print(f'U-Net initialized in {tf:.2f} secs')
    return unet_model

In [9]:
def load_unet(unet_path = UNET_PATH, device = None, use_safetensors = False) -> UNetModel:
    device = get_device()
    if use_safetensors:
        unet_path = unet_path.replace('.ckpt', '.safetensors')
        with section(f"U-Net model loading, from {unet_path}"):
            unet_dict = load_file(unet_path); print(unet_dict.keys())
            unet_model = initialize_unet(device=device)
            # load_model(unet_model, unet_path, strict=False)
            loaded_weights = load_file(unet_path, device='cpu')
            unet_model.load_state_dict(loaded_weights, strict=False)

    else:
        with section(f"U-Net model loading, from {unet_path}"):
            unet_model = torch.load(unet_path, map_location=device).eval()

    return unet_model

In [10]:
st_unet = load_unet(**path_tree.unet, device = 'cuda:0', use_safetensors=True)

INFO: `device` is None. Using device  cuda:0.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Starting section: U-Net model loading, from /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/unet/unet.safetensors...
dict_keys(['conv_in.bias', 'conv_in.weight', 'conv_norm_out.bias', 'conv_norm_out.weight', 'conv_out.bias', 'conv_out.weight', 'down_blocks.0.attentions.0.norm.bias', 'down_blocks.0.attentions.0.norm.weight', 'down_blocks.0.attentions.0.proj_in.bias', 'down_blocks.0.attentions.0.proj_in.weight', 'down_blocks.0.attentions.0.proj_out.bias', 'down_blocks.0.attentions.0.proj_out.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.bias', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.weight', 'down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.weight', 'down_blocks.0.attentions.0.transformer_bloc

In [11]:
ckpt_unet = load_unet(**path_tree.unet, device='cuda:0', use_safetensors=False)

INFO: `device` is None. Using device  cuda:0.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Starting section: U-Net model loading, from /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/unet/unet.ckpt...
Finished section: U-Net model loading, from /home/izzortsi/.cloned/kcg-ml-sd1p4/input/model/unet/unet.ckpt in 0.89 seconds

